In [1]:
!pip install pydantic
!pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 9.6 MB/s eta 0:00:00


In [2]:
import enum
import hashlib
import re
from typing import Any, Self
from pydantic import (BaseModel,EmailStr,Field,field_serializer,field_validator,model_serializer,model_validator,SecretStr)

In [3]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

In [5]:
class Role(enum.IntFlag): # definindo papeis disponíveis de usuários
    User = 0 # novo defalt
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

In [6]:
class User(BaseModel):
    name: str = Field(examples=["Example"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True, # adição de imutabilidade em campo name
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user", exclude=True # adição de parametro exclude, para ocultar senha durante serialização
    )
    role: Role = Field(
        description="The role of the user",
        examples=[1, 2, 4, 8],
        default=0,
        validate_default=True, # adição de parametro para validar valor padrão, como se tivesse sido feito pelo usuário
    )

    @field_validator("name")
    def validate_name(cls, v: str) -> str: # função de validação de name por expressão regular
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role: # função de validação de role
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user_pre(cls, v: dict[str, Any]) -> dict[str, Any]: # função de validaçaõ de nome, senha e função hash para mascarar senha
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

    @model_validator(mode="after")
    def validate_user_post(self, v: Any) -> Self: # validação de papel por nome
        if self.role == Role.Admin and self.name != "Arjan": # restringir acesso a user especifico
            raise ValueError("Only Arjan can be an admin")
        return self

    @field_serializer("role", when_used="json")
    @classmethod
    def serialize_role(cls, v) -> str: # controlar coversão do role em json quando houver a serialização
        return v.name

    @model_serializer(mode="wrap", when_used="json") #
    def serialize_user(self, serializer, info) -> dict[str, Any]: # verifica se filtros include ou exclude foram passados na hora da serialização e decide como gerar o JSON.
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)

# Função de Teste

Essa função cria um objeto User a partir do input "data", validando-os conforme as regras definidas na classe User, em seguida, demonstra diferentes formas de serializar e exibir esse objeto, seja como dicionário Python ou como string JSON, com ou sem aplicação de filtros.

In [7]:
def main() -> None:
    data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
        "role": "Admin",
    }
    user = User.model_validate(data)
    if user:
        print(
            "The serializer that returns a dict:",
            user.model_dump(),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a JSON string:",
            user.model_dump(mode="json"),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a json string, excluding the role:",
            user.model_dump(exclude=["role"], mode="json"),
            sep="\n",
            end="\n\n",
        )
        print("The serializer that encodes all values to a dict:", dict(user), sep="\n")

1. Dicionário python
2. json customizado básico, retornando apenas 'name' e 'role', pois identifiou a ausencia de filtros exclude ou include
3. json exluindo role
4. dicionário retornando todos valores incluindo, incluive a secha que era excluida por padrão

In [8]:
if __name__ == "__main__":
    main()

The serializer that returns a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'role': <Role.Admin: 4>}

The serializer that returns a JSON string:
{'name': 'Arjan', 'role': 'Admin'}

The serializer that returns a json string, excluding the role:
{'name': 'Arjan', 'email': 'example@arjancodes.com'}

The serializer that encodes all values to a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>}
